In [ ]:
!pip install mplfinance matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install pyts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
import os
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from pyts.image import GramianAngularField
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold


import shutil

In [ ]:
file_path = "/content/nba.csv"
data = pd.read_csv(file_path)

data.columns

Index(['GAME_ID', 'GAME_DATE', 'TEAM_ID', 'TEAM_ABBREVIATION', 'OPP_TEAM_ID',
       'OPP_TEAM_ABBREVIATION', 'PLUS_MINUS', 'WL', 'P_W_PCT', 'PAST_5',
       'PAST_5_PTS', 'P_W-L', 'P_W_PCT_OPP', 'PAST_5_OPP', 'PAST_5_PTS_OPP',
       'P_W-L_OPP'],
      dtype='object')

In [ ]:
data.head()

,GAME_ID,GAME_DATE,TEAM_ID,TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL,P_W_PCT,PAST_5,PAST_5_PTS,P_W-L,P_W_PCT_OPP,PAST_5_OPP,PAST_5_PTS_OPP,P_W-L_OPP
0,21900021,2019-10-25,1610612740,NOP,1610612742,DAL,-7,0,0.0,0.0,122.0,-1.0,1.0,1.0,108.0,1.0
1,21900023,2019-10-25,1610612743,DEN,1610612756,PHX,1,1,1.0,1.0,108.0,1.0,1.0,1.0,124.0,1.0
2,21900019,2019-10-25,1610612751,BKN,1610612752,NYK,4,1,0.0,0.0,126.0,-1.0,0.0,0.0,111.0,-1.0
3,21900024,2019-10-25,1610612758,SAC,1610612757,POR,-10,0,0.0,0.0,95.0,-1.0,0.0,0.0,100.0,-1.0
4,21900018,2019-10-25,1610612766,CHA,1610612750,MIN,-22,0,1.0,1.0,126.0,1.0,1.0,1.0,127.0,1.0


In [ ]:
# Convert GAME_DATE to datetime
data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'], format='%Y-%m-%d')

# Ensure data is sorted by team and date
data = data.sort_values(by=['TEAM_ID', 'GAME_DATE'])

In [ ]:
# data.to_csv('newnba.csv', index=False)
df = data

In [ ]:
# shutil.rmtree("candlestick_nba.csv", ignore_errors=True)

In [ ]:
# os.remove("candlestick_nba.csv")

The dataset includes key features like:

* PLUS_MINUS: A proxy for performance fluctuation.
* PAST_5, PAST_5_PTS: Recent trends in performance.
* P_W_PCT: Pre-game win percentage.
* PAST_5_OPP, PAST_5_PTS_OPP: Opponent's trends.

The rolling window approach reflects recent performance trends, critical for short-term sports predictions. Opponent-specific data (P_W_PCT_OPP, PAST_5_OPP) ensures the model considers competition strength.
#### 1. Derive Candlestick Components:

* Open (O): Use P_W_PCT or the team’s win percentage before the game.
* High (H): The highest performance metric, e.g., maximum PLUS_MINUS over the last 5 games.
* Low (L): The lowest performance metric, e.g., minimum PLUS_MINUS over the last 5 games.
* Close (C): The actual PLUS_MINUS for the game or binary win/loss (WL).

#### 2.Create Candlestick Data:
 * Aggregate the above values over a rolling window (e.g., last 5 games) to create time-series data for each team.

#### 3. Add Opponent-Specific Data:

* Include features like P_W_PCT_OPP and PAST_5_OPP to account for opponent strength.

In [ ]:
# candlestick_nba.csv

nba_data = data

# Create rolling window statistics for candlestick components
def calculate_candlestick_data(df):
    df['O'] = df['P_W_PCT']  # Opening odds or pre-game win percentage
    df['H'] = df['PLUS_MINUS'].rolling(window=5).max()  # Max of the last 5 games
    df['L'] = df['PLUS_MINUS'].rolling(window=5).min()  # Min of the last 5 games
    df['C'] = df['PLUS_MINUS']  # Closing is the actual performance in the game
    return df

# Apply the function to each team
nba_data = nba_data.groupby('TEAM_ID').apply(calculate_candlestick_data)

# Add opponent-specific data
nba_data['O_OPP'] = nba_data['P_W_PCT_OPP']
nba_data['H_OPP'] = nba_data['PAST_5_OPP']
nba_data['L_OPP'] = nba_data['PAST_5_PTS_OPP']

# Drop NaN rows caused by rolling operations
nba_data = nba_data.dropna(subset=['H', 'L'])

# Save the processed dataset
nba_data.to_csv('candlestick_nba.csv', index=False)

# Print a preview
nba_data[['GAME_ID', 'TEAM_ID', 'O', 'H', 'L', 'C', 'O_OPP', 'H_OPP', 'L_OPP']].head()

<ipython-input-9-682e5b80fc39>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nba_data = nba_data.groupby('TEAM_ID').apply(calculate_candlestick_data)


GAME_ID     TEAM_ID      O    H     L   C  O_OPP  H_OPP  \
TEAM_ID                                                                    
1610612737 91   21900105  1610612737  0.500  8.0 -20.0 -20  0.250    0.2   
           105  21900119  1610612737  0.429  8.0 -20.0 -12  0.250    0.4   
           192  21900204  1610612737  0.308  8.0 -20.0  -8  0.769    0.8   
           213  21900228  1610612737  0.267  8.0 -20.0  -3  0.714    0.6   
           227  21900243  1610612737  0.250 -3.0 -20.0 -12  0.500    0.2   

                L_OPP  
TEAM_ID                
1610612737 91   105.6  
           105  108.0  
           192  112.4  
           213  109.8  
           227  105.2

**Candlestick Chart Generation**

#### 1. Visualize Data as Candlestick Charts:

* Generate candlestick charts for each game or rolling sequence.
* Each chart represents a team's performance over time.

#### 2. Convert Charts to 2D Images:

* Save the generated candlestick charts as images for CNN processing.

By saving charts as images, we can leverage CNNs for pattern recognition directly.

In [ ]:
# Load processed candlestick data
file_path = 'candlestick_nba.csv'
nba_data = pd.read_csv(file_path)

# Create a directory to save the candlestick charts
output_dir = './candlestick_images/'
os.makedirs(output_dir, exist_ok=True)

# Normalize features for consistent scaling
scaler = MinMaxScaler(feature_range=(0, 1))

# Generate candlestick charts for each team
def generate_candlestick_charts(df, team_id):
    for i in range(5, len(df)):  # Rolling window (e.g., 5 games)
        rolling_data = df.iloc[i-5:i]  # Select the last 5 games
        candlestick_data = pd.DataFrame({
            'Date': pd.to_datetime(rolling_data['GAME_DATE']),
            'Open': rolling_data['O'],
            'High': rolling_data['H'],
            'Low': rolling_data['L'],
            'Close': rolling_data['C']
        })
        candlestick_data.set_index('Date', inplace=True)

        # Normalize the data for consistent chart scaling
        normalized_data = scaler.fit_transform(candlestick_data)
        candlestick_data[['Open', 'High', 'Low', 'Close']] = normalized_data

        # Use the last game's ID for the file name
        game_id = rolling_data.iloc[-1]['GAME_ID']
        chart_title = f'Team_{team_id}_Game_{game_id}'
        save_path = os.path.join(output_dir, f'{chart_title}.png')
        mpf.plot(candlestick_data, type='candle', style='charles', title=chart_title, savefig=save_path)

# Group by team and generate charts
for team_id, team_data in nba_data.groupby('TEAM_ID'):
    generate_candlestick_charts(team_data, team_id)

print(f"Candlestick charts saved to {output_dir}")

Candlestick charts saved to ./candlestick_images/


Trying another way

**Gramian Angular Field** (GAF) encoding enhances the representation by converting raw time-series data into structured visual formats, potentially improving model performance.

In [ ]:
# Create a directory to save the candlestick charts
gaf_output_dir = './gaf_candlestick_images/'
os.makedirs(gaf_output_dir, exist_ok=True)

# Convert rolling data into GAF images
def generate_gaf_images(df, team_id, gaf_output_dir):
    gaf = GramianAngularField(image_size=5, method='summation')  # Match the rolling window size
    for i in range(5, len(df)):  # Rolling window of 5 games
        rolling_data = df.iloc[i-5:i][['O', 'H', 'L', 'C']].values  # Extract OHLC data
        normalized_data = scaler.fit_transform(rolling_data)  # Normalize to [0, 1]

        # Flatten the rolling data for GAF processing
        gaf_image = gaf.fit_transform([normalized_data.flatten()])[0]  # Create GAF image

        # Use the last game's ID for the file name
        game_id = df.iloc[i]['GAME_ID']
        image_title = f'Team_{team_id}_Game_{game_id}_GAF.png'
        plt.imsave(os.path.join(gaf_output_dir, image_title), gaf_image, cmap='rainbow')

# Group by team and generate GAF images
for team_id, team_data in nba_data.groupby('TEAM_ID'):
    generate_gaf_images(team_data, team_id, gaf_output_dir)

print(f"GAF images saved to {gaf_output_dir}")

GAF images saved to ./gaf_candlestick_images/


Creating Labels

In [ ]:
# Path to the candlestick images and labels
image_dir = './candlestick_images/'
label_file = 'candlestick_nba.csv'

# Load labels (WL column as target variable)
labels_data = pd.read_csv(label_file)
labels_data = labels_data[['GAME_ID', 'WL', 'TEAM_ID']]

# Associate each image with its label
image_paths = []
labels = []
for filename in os.listdir(image_dir):
    if filename.endswith('.png'):
        game_id = int(filename.split('.')[0].split('_')[3])  # Extract GAME_ID from the filename
        label = labels_data[labels_data['GAME_ID'] == game_id]['WL'].values[0]
        image_paths.append(os.path.join(image_dir, filename))
        labels.append(label)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

**CNN w/ Candlestick**

Test Accuracy: 77.42%

In [ ]:
# Load images and preprocess them
def load_and_preprocess_images(image_paths):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=(64, 64))  # Resize to 64x64 for CNN
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img_array)
    return np.array(images)

X_train = load_and_preprocess_images(train_paths)
X_test = load_and_preprocess_images(test_paths)
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

In [ ]:
# Define CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 282ms/step - accuracy: 0.4491 - loss: 1.9196 - val_accuracy: 0.5355 - val_loss: 0.6904
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 200ms/step - accuracy: 0.5529 - loss: 0.6903 - val_accuracy: 0.6516 - val_loss: 0.6854
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step - accuracy: 0.6031 - loss: 0.6694 - val_accuracy: 0.6774 - val_loss: 0.6375
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - accuracy: 0.6965 - loss: 0.5912 - val_accuracy: 0.6903 - val_loss: 0.5718
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - accuracy: 0.7401 - loss: 0.5248 - val_accuracy: 0.7355 - val_loss: 0.5478
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 277ms/step - accuracy: 0.8102 - loss: 0.4703 - val_accuracy: 0.7419 - val_loss: 0.5335
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 193ms/step - accuracy: 0.7993 - loss: 0.4462 - val_accuracy: 0.7613 - val_loss: 0.5175
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.8346 - loss: 0.4082 - val_accuracy: 0.7742 - val_

**K-fold**

Mean Accuracy: 80.98%


Standard Deviation of Accuracy: 2.06%

In [ ]:
# Define the CNN model as a function for reuse
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Output layer for binary classification
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load your image data and labels (from previous preprocessing)
X = load_and_preprocess_images(image_paths)  # Assuming you preprocessed images earlier
y = to_categorical(labels)  # Convert labels to one-hot encoding

# Define K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_index, val_index in kf.split(X):
    print(f"Training on fold {fold_no}...")

    # Split the data into training and validation sets for this fold
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create a new CNN model for each fold
    model = create_cnn_model()

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold_no} Validation Accuracy: {val_accuracy * 100:.2f}%")

    # Save the accuracy
    accuracies.append(val_accuracy)

    fold_no += 1

# Calculate and print the mean and standard deviation of accuracies
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"Mean Accuracy: {mean_accuracy * 100:.2f}%")
print(f"Standard Deviation of Accuracy: {std_accuracy * 100:.2f}%")

Training on fold 1...
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 238ms/step - accuracy: 0.5386 - loss: 1.2206 - val_accuracy: 0.5355 - val_loss: 0.6925
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.5341 - loss: 0.6914 - val_accuracy: 0.5548 - val_loss: 0.6898
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.5730 - loss: 0.6793 - val_accuracy: 0.6710 - val_loss: 0.6406
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 233ms/step - accuracy: 0.6459 - loss: 0.6227 - val_accuracy: 0.6710 - val_loss: 0.6068
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.7405 - loss: 0.5559 - val_accuracy: 0.7677 - val_loss: 0.5388
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.7683 - loss: 0.4920 - val_accuracy: 0.7226 - val_loss: 0.5335
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.8195 - loss: 0.4515 - val_accuracy: 0.7677 - val_loss: 0.5139
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 177ms/step - accuracy: 0.8122 - loss: 0.4

**Learning Rate Scheduling** to adjust learning rates dynamically during training.

Exponential Decap

One run: 81%

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Define the CNN model
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Output layer for binary classification
    ])
    return model

# Define exponential decay function
def exponential_decay(epoch, lr):
    initial_lr = 0.001  # Initial learning rate
    k = 0.1  # Decay factor
    new_lr = initial_lr * np.exp(-k * epoch)
    return new_lr

# Instantiate the model
model = create_cnn_model()

# Compile the model with an Adam optimizer
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(exponential_decay)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler],
    verbose=1
)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 427ms/step - accuracy: 0.4810 - loss: 1.2086 - val_accuracy: 0.5355 - val_loss: 0.6913 - learning_rate: 0.0010
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 195ms/step - accuracy: 0.4871 - loss: 0.6911 - val_accuracy: 0.5355 - val_loss: 0.6862 - learning_rate: 9.0484e-04
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step - accuracy: 0.5384 - loss: 0.6822 - val_accuracy: 0.7032 - val_loss: 0.6662 - learning_rate: 8.1873e-04
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 250ms/step - accuracy: 0.6265 - loss: 0.6601 - val_accuracy: 0.7097 - val_loss: 0.6239 - learning_rate: 7.4082e-04
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - accuracy: 0.6826 - loss: 0.6109 - val_accuracy: 0.7290 - val_loss: 0.5778 - learning_rate: 6.7032e-04
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 185ms/step - accuracy: 0.7405 - loss: 0.5487 - val_accuracy: 0.7677 - val_loss: 0.5371 - learning_rate: 6.0653e-04
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 296ms/step - accuracy: 0.7

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

5/5 - 1s - 113ms/step - accuracy: 0.8129 - loss: 0.4328
Test Accuracy: 81.29%


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to generate candlestick features based on rolling window size
def generate_candlestick_features(data, window_size):
    data['O'] = data['P_W_PCT']
    data['H'] = data['PLUS_MINUS'].rolling(window=window_size).max()
    data['L'] = data['PLUS_MINUS'].rolling(window=window_size).min()
    data['C'] = data['PLUS_MINUS']
    return data.dropna(subset=['H', 'L'])  # Drop NaN rows caused by rolling

# CNN model function
def create_cnn_model(input_shape=(64, 64, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to preprocess images
def preprocess_images(image_paths):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=(64, 64))  # Resize to 64x64
        img_array = img_to_array(img) / 255.0  # Normalize
        images.append(img_array)
    return np.array(images)

# Cross-validation to find the best rolling window size
def find_best_rolling_window(data, labels, window_sizes, k=5):
    best_window = None
    best_accuracy = 0.0
    results = []

    for window_size in window_sizes:
        print(f"Evaluating window size: {window_size}")

        # Generate candlestick features
        candlestick_data = generate_candlestick_features(data.copy(), window_size)

        # Prepare image paths and labels
        image_paths = candlestick_data['image_paths'].values  # Assume pre-generated candlestick images
        y = to_categorical(labels)  # One-hot encode labels

        # Preprocess images
        X = preprocess_images(image_paths)

        # K-Fold cross-validation
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        fold_accuracies = []

        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            # Create and train the model
            model = create_cnn_model(input_shape=(64, 64, 3))
            model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Use fewer epochs for faster testing

            # Evaluate on validation set
            _, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            fold_accuracies.append(val_accuracy)

        # Average accuracy across folds
        avg_accuracy = np.mean(fold_accuracies)
        print(f"Window size {window_size}: Average Accuracy = {avg_accuracy:.4f}")
        results.append((window_size, avg_accuracy))

        # Update best window size
        if avg_accuracy > best_accuracy:
            best_window = window_size
            best_accuracy = avg_accuracy

    print(f"Best Window Size: {best_window} with Accuracy: {best_accuracy:.4f}")
    return best_window, results

# Example usage
window_sizes = [3, 5, 7, 10]
data = pd.read_csv('candlestick_nba.csv')  # Load your processed dataset
labels = data['WL'].values  # Target variable (win/loss)

best_window, results = find_best_rolling_window(data, labels, window_sizes)

# Display results
for window, acc in results:
    print(f"Window Size: {window}, Accuracy: {acc:.4f}")


Evaluating window size: 3


KeyError: 'image_paths'

**GAM**


Test Accuracy: 48.39%

In [ ]:
# Path to GAF images and labels
gaf_image_dir = './gaf_candlestick_images/'
label_file = 'candlestick_nba.csv'

# Load labels (WL column as target variable)
labels_data = pd.read_csv(label_file)
labels_data = labels_data[['GAME_ID', 'WL', 'TEAM_ID']]

# Associate each GAF image with its label
image_paths = []
labels = []
for filename in os.listdir(gaf_image_dir):
    if filename.endswith('.png'):
        game_id = int(filename.split('.')[0].split('_')[3]) # Extract GAME_ID from filename
        label = labels_data[labels_data['GAME_ID'] == game_id]['WL'].values[0]
        image_paths.append(os.path.join(gaf_image_dir, filename))
        labels.append(label)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Train-Test Split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

# Load GAF images and preprocess
def load_and_preprocess_images(image_paths):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=(32, 32))  # GAF images are 32x32
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img_array)
    return np.array(images)

X_train = load_and_preprocess_images(train_paths)
X_test = load_and_preprocess_images(test_paths)
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

# Define the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.5066 - loss: 0.7811 - val_accuracy: 0.4839 - val_loss: 0.7011
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.5210 - loss: 0.6936 - val_accuracy: 0.4839 - val_loss: 0.6976
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.5262 - loss: 0.6905 - val_accuracy: 0.4839 - val_loss: 0.7016
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5650 - loss: 0.6963 - val_accuracy: 0.4968 - val_loss: 0.6935
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.5430 - loss: 0.6882 - val_accuracy: 0.4839 - val_loss: 0.6953
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.5243 - loss: 0.6883 - val_accuracy: 0.4839 - val_loss: 0.6965
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5463 - loss: 0.6891 - val_accuracy: 0.4903 - val_loss: 0.6961
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5734 - loss: 0.6842 - val_accuracy: 0.5032 - val_loss: 0.

In [ ]:
# # Don't use data augmentation.. not smart idea

# datagen = ImageDataGenerator(
#     rotation_range=15,         # Rotate images up to 15 degrees
#     width_shift_range=0.1,     # Shift image width up to 10%
#     height_shift_range=0.1,    # Shift image height up to 10%
#     horizontal_flip=True,      # Randomly flip images horizontally
#     fill_mode='nearest'        # Fill pixels after transformations
# )

# # Apply the augmentation only to the training dataset
# datagen.fit(X_train)

# # Use the augmented data generator during training
# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=32),  # Augmented training data
#     epochs=20,                                     # Train for 20 epochs
#     validation_data=(X_test, y_test),              # Validation remains unchanged
# )


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.5628 - loss: 0.6778 - val_accuracy: 0.5484 - val_loss: 0.6923
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5809 - loss: 0.6852 - val_accuracy: 0.5355 - val_loss: 0.6929
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.5428 - loss: 0.6843 - val_accuracy: 0.5484 - val_loss: 0.6898
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.5891 - loss: 0.6796 - val_accuracy: 0.5290 - val_loss: 0.6889
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.5411 - loss: 0.6914 - val_accuracy: 0.5613 - val_loss: 0.6920
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5348 - loss: 0.6880 - val_accuracy: 0.4903 - val_loss: 0.6941
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.5376 - loss: 0.6845 - val_accuracy: 0.5355 - val_loss: 0.6903
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5670 - loss: 0.6823 - val_accuracy: 0.4968 - val_loss: 0.